In [2]:
import math
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt

import roboticstoolbox as rtb
import spatialmath as sm # provides objects for representing transformations
from swift import Swift # lightweight browser-based simulator which comes with the toolbox
import spatialgeometry as sg # utility package for dealing with geometric objects

In [124]:
robot = rtb.models.wx200()
print(robot)
useful_links = lambda robot: [robot.links[i] for i in (0,1,2,3,4)]

ERobot: wx200 (by Interbotix), 8 joints (RRRRRRPP), 4 branches, dynamics, geometry, collision
┌──────┬─────────────────────┬───────┬───────────────────┬───────────────────────────────────────────┐
│ link │        link         │ joint │      parent       │            ETS: parent to link            │
├──────┼─────────────────────┼───────┼───────────────────┼───────────────────────────────────────────┤
│    0 │ /base_link          │       │ BASE              │ SE3()                                     │
│    1 │ /shoulder_link      │     0 │ /base_link        │ SE3(0, 0, 0.0716) ⊕ Rz(q0)                │
│    2 │ /upper_arm_link     │     1 │ /shoulder_link    │ SE3(0, 0, 0.03865) ⊕ Ry(q1)               │
│    3 │ /forearm_link       │     2 │ /upper_arm_link   │ SE3(0.05, 0, 0.2; 180°, -0°, 0°) ⊕ Ry(q2) │
│    4 │ /wrist_link         │     3 │ /forearm_link     │ SE3(0.2, 0, 0) ⊕ Ry(q3)                   │
│    5 │ /gripper_link       │     4 │ /wrist_link       │ SE3(0.065, 0, 0; -180°,

In [125]:
def has_pairwise_collision(links):
	n_links = len(links)
	for i1 in range(n_links):
		link1 = links[i1]
		for i2 in list(range(0,i1-1))+list(range(i1+2,n_links)):
			link2 = links[i2]
			for collider1 in link1.collision:
				if collider1.stype == 'mesh':
					continue
				for collider2 in link2.collision:
					if collider2.stype == 'mesh':
						continue
					if collider1.iscollided(collider2):
						print(collider1, '\n with \n', collider2)
						return True

	return False

In [126]:
has_pairwise_collision(useful_links(robot))

False

In [102]:
# Make the environment
env = Swift()

# Launch the simulator, will open a browser tab in your default
# browser (chrome is recommended)
# The realtime flag will ask the simulator to simulate as close as
# possible to realtime as apposed to as fast as possible
# We can also choose to see Swift within the notebook using the
# browser="notebook" flag
env.launch(realtime=True, browser="notebook")

# obstacle = rtb.tools.urdf.Box([8, 1, 1])#, sm.SE3(1, 0, 0))

# We can then add our robot to the simulator envionment
env.add(ob=robot)
# env.add(obstacle)

# end-effector axes
ee_axes = sg.Axes(0.1)

# goal axes
goal_axes = sg.Axes(0.1)

# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes) 

2

In [5]:
# We can set the pose of the shape using the `.T` attribute.
# This pose can be either a spatialmat SE3 or a 4x4 ndarray
ee_axes.T = robot.fkine(robot.q, end='/ee_arm_link')

# Set the goal axes to something along the x axis
goal_axes.T = sm.SE3.Trans(0.5, 0.0, 0.5)

# step the environment to view the changes
env.step(0.05)